## JSON

In [3]:
import json  
from pprint import pprint

Для выполнения десериализации мы воспользуемся методом load модуля json. В качестве параметра укажем ссылку на файл:

In [19]:
with open('data/recipes.json') as f:
    recipes = json.load(f)

def get_recipe_by_id(id):
    for recipe in recipes:
        if recipe['id'] == id:
            return recipe

# pprint(recipes)

print(type(recipes))
print(len(recipes))

<class 'list'>
500


Какую кухню (ключ 'cuisine') представляет последнее блюдо в списке recipes?

In [13]:
recipes[-1]['cuisine']

'british'

Сколько ингредиентов входит в состав этого блюда?

In [15]:
len(recipes[-1]['ingredients'])

17

Какие из перечисленных ингредиентов присутствуют в блюде с id = 17636?

In [20]:
get_recipe_by_id(17636)['ingredients']

['tomato sauce',
 'shredded carrots',
 'spinach',
 'part-skim mozzarella cheese',
 'italian seasoning',
 'english muffins, split and toasted',
 'chopped onion',
 'vegetable oil cooking spray',
 'chopped green bell pepper']

Сколько ингредиентов входит в состав рецепта для приготовления блюда с id = 42013?

In [21]:
len(get_recipe_by_id(42013)['ingredients'])

14

Какие ингредиенты отсутствуют в блюде с id = 23629?

In [22]:
get_recipe_by_id(23629)['ingredients']

['eggs',
 'russet potatoes',
 'mexican chorizo',
 'black beans',
 'salsa',
 'canola oil',
 'colby cheese',
 'black olives',
 'sour cream',
 'avocado',
 'half & half',
 'goat cheese']

Сколько ингредиентов включено в состав всех блюд, описанных в наборе данных?

In [24]:
ingredients = set()
for recipe in recipes:
    ingredients.update(recipe['ingredients'])
print(len(ingredients))

1318


Сколько ингредиентов встречается в рецептах блюд итальянской кухни ('cuisine' = italian)?

In [25]:
italian_ingredients = set()
for recipe in recipes:
    if recipe['cuisine'] == 'italian':
        italian_ingredients.update(recipe['ingredients'])
print(len(italian_ingredients))

406


Какие ингредиенты не встречаются в рецептах блюд русской кухни ('cuisine' = 'russian')?

In [26]:
russian_ingredients = set()
for recipe in recipes:
    if recipe['cuisine'] == 'russian':
        russian_ingredients.update(recipe['ingredients'])
print(russian_ingredients)

{'red beets', 'onions', 'cucumber', 'dill', 'mozzarella cheese', 'grits', 'water', 'salt', 'sugar', 'boiled eggs', 'buttermilk'}


Давайте представим, что мы хотим заняться торговлей продуктами для ресторанов национальной кухни. Нам необходимо оценить, какие продукты используются в приготовлении блюд чаще, а какие — реже. Давайте попробуем решить эту задачу с использованием имеющегося у нас набора данных.

In [29]:
food = {}
for item in ingredients:
   food[item] = 0
for recipe in recipes:
    for item in recipe['ingredients']:
        food[item] += 1

Какие ингредиенты входят в состав более чем 20% блюд, представленных в наборе данных?

Подсказка: Напоминаем, что весь набор данных содержит сведения о 500 блюдах.

In [30]:
print(food['water'] > 100)
print(food['olive oil'] > 100)
print(food['salt'] > 100)
print(food['garlic'] > 100)
print(food['pepper'] > 100)

False
False
True
True
False


Какой ингредиент входит в состав самого большого количества блюд?

In [37]:
most_popular = None
dishes_count = 0
for ingredient in food:
    if food[ingredient] > dishes_count:
        most_popular = ingredient
        dishes_count = food[most_popular]

most_popular

'salt'

Сколько ингредиентов входит в состав только одного блюда?

In [39]:
singletons = 0
for ingredient in food:
    if food[ingredient] == 1:
        singletons += 1

singletons

684

In [56]:
import pandas as pd
df = pd.DataFrame(recipes)
print(df.info())
df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           500 non-null    int64 
 1   cuisine      500 non-null    object
 2   ingredients  500 non-null    object
dtypes: int64(1), object(2)
memory usage: 11.8+ KB
None


,id,cuisine,ingredients
0,10259,greek,"[romaine lettuce, black olives, grape tomatoes..."
1,25693,southern_us,"[plain flour, ground pepper, salt, tomatoes, g..."
2,20130,filipino,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
3,22213,indian,"[water, vegetable oil, wheat, salt]"
4,13162,indian,"[black pepper, shallots, cornflour, cayenne pe..."


Работу над созданием датафрейма мы начнём с создания и заполнения столбцов, содержащих сведения о наличии или отсутствии каждого ингредиента в рецепте. Процесс заполнения  будем проводить в два этапа: 

1. Создадим функцию для заполнения значения в каждой ячейке. Функция будет проверять наличие конкретного ингредиента в столбце 'ingredients' для текущего блюда и возвращать 1, если ингредиент есть в рецепте, и 0, если он отсутствует.
2. Организуем цикл с параметром, в котором будем перебирать наименования всех ингредиентов. Для каждого ингредиента создадим в датафрейме столбец с соответствующим названием и заполним его единичками и нулями, применив к датафрейму, а точнее, к столбцу 'ingredients', функцию, созданную нами на предыдущем шаге.

Код функции, с помощью которой мы будем создавать новые столбцы и заполнять их значениями, могут выглядеть следующим образом:

In [58]:
def find_item(cell):
    if item in cell:
        return 1
    return 0

Теперь перейдём к созданию кода, в котором будут создаваться столбцы с информацией о наличии или отсутствии того или иного ингредиента в конкретном рецепте:

In [59]:
for item in ingredients:
    df[item] = df['ingredients'].apply(find_item)
df.head()

,id,cuisine,ingredients,diced tomatoes,great northern beans,shredded cheese,unsalted dry roast peanuts,bread slices,zinfandel,reduced sodium soy sauce,...,white radish,ranch dressing,spinach,rice cakes,wasabi,radicchio,roasted garlic,spinach leaves,iceberg lettuce,dried tart cherries
0,10259,greek,"[romaine lettuce, black olives, grape tomatoes...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,25693,southern_us,"[plain flour, ground pepper, salt, tomatoes, g...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,20130,filipino,"[eggs, pepper, salt, mayonaise, cooking oil, g...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,22213,indian,"[water, vegetable oil, wheat, salt]",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,13162,indian,"[black pepper, shallots, cornflour, cayenne pe...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


В завершение изменим значение столбца ingredients, заполнив его данными о количестве ингредиентов в каждом рецепте:

In [60]:
df['ingredients'] = df['ingredients'].apply(lambda x: len(x))
df.head()

,id,cuisine,ingredients,diced tomatoes,great northern beans,shredded cheese,unsalted dry roast peanuts,bread slices,zinfandel,reduced sodium soy sauce,...,white radish,ranch dressing,spinach,rice cakes,wasabi,radicchio,roasted garlic,spinach leaves,iceberg lettuce,dried tart cherries
0,10259,greek,9,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,25693,southern_us,11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,20130,filipino,12,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,22213,indian,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,13162,indian,20,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [62]:
import pandas as pd

def find_item(cell):
    if item in cell:
        return 1
    return 0

with open('data/recipes.json') as f:
    recipes = json.load(f)
    
ingredients = set()
for recipe in recipes:
    ingredients.update(recipe['ingredients'])

df = pd.DataFrame(recipes)

for item in ingredients:
    df[item] = df['ingredients'].apply(find_item)

df['ingredients'] = df['ingredients'].apply(lambda x: len(x))

df.head()

,id,cuisine,ingredients,diced tomatoes,great northern beans,shredded cheese,unsalted dry roast peanuts,bread slices,zinfandel,reduced sodium soy sauce,...,white radish,ranch dressing,spinach,rice cakes,wasabi,radicchio,roasted garlic,spinach leaves,iceberg lettuce,dried tart cherries
0,10259,greek,9,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,25693,southern_us,11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,20130,filipino,12,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,22213,indian,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,13162,indian,20,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [64]:
df.to_csv('data/recipes.csv', index = False)

## Из Pandas в JSON

In [100]:
df = pd.read_csv('data/recipes.csv')

In [101]:
df.head()

,id,cuisine,ingredients,diced tomatoes,great northern beans,shredded cheese,unsalted dry roast peanuts,bread slices,zinfandel,reduced sodium soy sauce,...,white radish,ranch dressing,spinach,rice cakes,wasabi,radicchio,roasted garlic,spinach leaves,iceberg lettuce,dried tart cherries
0,10259,greek,9,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,25693,southern_us,11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,20130,filipino,12,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,22213,indian,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,13162,indian,20,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Напишите код для создания списка id всех блюд, представленных в датафрейме.

In [102]:
ids = list(df.id.values)

Напишите код для создания списка ингредиентов всех блюд, представленных в датафрейме.



In [103]:
ingredients = list(df.columns[3:].values)

In [119]:
def make_list(df):
    s = df.iloc[0]
    return list(s[s.eq(1)].index.values)

new_recipes = []

for current_id in ids:
    cuisine = df[df['id'] == current_id]['cuisine'].iloc[0]
    current_ingredients = make_list(df[df['id'] == current_id])
    current_recipe = {'cuisine': cuisine, 'id': int(current_id), 'ingredients': current_ingredients}
    new_recipes.append(current_recipe)

В завершающей части данного блока мы выполним сериализацию списка new_recipes и запишем полученные данные в файл. Для сериализации мы будем использовать функцию dumps(), которой в качестве параметра передадим список new_recipes. Запись в файл осуществляется с помощью метода write(). Предварительно файл нужно будет открыть для записи с помощью функции open и параметра 'w':

In [120]:
new_recipes = json.dumps(new_recipes)
with open("data/new_recipes.json", "w") as write_file:
    write_file.write(new_recipes)

Оба действия можно совместить, если использовать метод dump():

In [121]:
with open("data/new_recipes.json", "w") as write_file:
    json.dump(new_recipes, write_file)